# prepare data for processing

In [37]:
# prepare data for processing
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
#define similariry function 
import numpy as np
import pandas as pd
import math
import re
import time

def dotproduct_(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def dotproduct(v1, v2):
    tmp = 0
    for a, b in zip(v1, v2):
        if a >= 0 and b >= 0:
            tmp += (a * b)
    return tmp   


def length(v):
    return math.sqrt(dotproduct(v, v))

def similar(v1, v2):
    return (dotproduct(v1, v2) / (length(v1) * length(v2)))

def length2(v1, v2):
    return math.sqrt(sum((a-b)**2 for a, b in zip(v1, v2)))
def length3(v1, v2):
    tmp = 0
    for a, b in zip(v1, v2):
        if a >= 0 and b >= 0:
            tmp += (a - b)**2
    return math.sqrt(tmp)       

#remove accents vietnamese
def remove_accents(text):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
#     s1.encode('utf-8')
#     s0.encode('utf-8')
    if text == np.nan or text == np.NAN or text == np.NaN:
        return text
    else:    
        s = ''
        text = str(text)
        text = text.decode('utf-8')

        for c in text:
            if c in s1:
                s += s0[s1.index(c)]
            else:
                s += c
        return s.strip()
    
# remove_accents('Hỗ trợ 4G hôm nãy đi chợi Hỗ trợ Hỗ trợ')   


def list_similar(item, numofpro, dataframe):
    list_similarity = dataframe[str(item)].sort(inplace = False)[: numofpro + 1]# sort by distance
    if str(item) in list_similarity:
        return list_similarity.drop([str(item)])
    #item_similarity = list_similarity.index.tolist()
    #item_similarity = map(lambda x: np.int(x),item_similarity)
    #del list_similarity[list_similarity.index(item)]
    else:
        return list_similarity#, item_similarity[1: numofpro+1]

def list_best_sale(category, numofpro, dataframe):
    #dataframe = dataframe.fillna(0)
#     best_sell = dataframe.loc[np.int(category)].sort_values(ascending = False, inplace = False)
#     item_best_sell = best_sell.index.tolist()
#     return item_best_sell[1: numofpro+1]# , best_sell
    return dataframe.loc[category].sort_values(ascending = False, inplace = False)[0: numofpro]

In [38]:
#read dataset and normalize
# data = pd.read_csv('/home/tgdd/TA_69/recommenderr/VuiVuiPropertyData.txt',sep='\t', delimiter='\t',  header =0, encoding='utf8', engine='python')
data = pd.read_excel('/home/tgdd/TA_69/recommenderr/1.xlsx', sheetname='Sheet1', header = 0)
new_col = map(remove_accents, list(data.columns))
data.columns = new_col
data = data.applymap(remove_accents)
data = data.dropna(axis = 1, how = 'all')
data = data.dropna(axis = 0, how = 'all')
data = data.drop_duplicates(['Ma san pham'], keep='last')

data = data[data['Gia'] != '-1']
data = data[data['Gia'] != '0']
# data = data[data.CATE.isin(['2423','2424', '2425','2426','2514','2515','2644','2663'])]

# set rule (price, branch) for all product

In [39]:
def rule_gia_all(var):
    max_price = data_final.Gia[data_final.Category == Cate].max()
    min_price = data_final.Gia[data_final.Category == Cate].min()
    tmp_price = (max_price - min_price)/10
    
    if var < min_price + tmp_price*1:
        val = 1
    elif var < min_price + tmp_price*2:
        val = 2
    elif var < min_price + tmp_price*3:
        val = 3
    elif var < min_price + tmp_price*4:
        val = 4
    elif var < min_price + tmp_price*5:
        val = 5
    elif var < min_price + tmp_price*6:
        val = 6
    elif var < min_price + tmp_price*7:
        val = 7
    elif var < min_price + tmp_price*8:
        val = 8
    elif var < min_price + tmp_price*9:
        val = 9
    else:
        val = 10
    return val + int(Cates.index(Cate) * 100)# + int(CATES.index(CATE) *100000)

In [32]:
#Moi CATE 1000
#Moi cate 100
# data_final: all category have less feature
data_final = data.icol([0, 1, 2, 3, 4, 5])
data_final.columns = ['Ma_san_pham', 'Category','CATE','Ten_san_pham', 'Hang_san_xuat','Gia']
data_final = data_final[~data_final.Category.isin(['42','44', '2002','1942','1943','1944','1962','1992', '1988', '382', '55', '54', '57', '58'])]
#data.Category[data.Category == 4444] = 2
data_final['CATE2'] = -1
data_final['Category2'] = -2
data_final.Gia = data_final.Gia.apply(lambda x: np.float(x))
data_final['Gia2'] = data_final['Gia']
CATES = data_final['CATE'].unique().tolist()
for CATE in CATES:
    data_final.CATE2[data_final.CATE == CATE] = int(CATES.index(CATE) *100000)
    Cates = data_final.Category[data_final['CATE'] == CATE].unique().tolist()#
    for Cate in Cates:
        data_final.Category2[data_final.Category == Cate] = int(Cates.index(Cate) * 100) + int(CATES.index(CATE) *100000)
        max_price = data_final.Gia[data_final.Category == Cate].max()
        min_price = data_final.Gia[data_final.Category == Cate].min()
        tmp_price = (max_price - min_price)/10
       
        data_final.Gia2[data_final.Category == Cate] = data_final.Gia2[data_final.Category == Cate].apply(lambda var: rule_gia_all(var))
        Hang_tmp = data_final.Hang_san_xuat[data_final.Category== Cate].tolist()
        data_final.Hang_san_xuat[data_final.Category== Cate] = data_final.Hang_san_xuat[data_final.Category== Cate].apply(lambda x: Hang_tmp.index(x)) + int(Cates.index(Cate) * 100) + int(CATES.index(CATE) *100000)
        #mask = data_final.Gia[data_final.Category == '42'].apply(lambda x: int(x)) > 1000000.
            #data_final.Gia2[mask] = percent + data_final.Category2[data_final.Category == Cate]

#df['Hang san xuat2'] = df.apply(rule_hang_ml, axis = 1)

#lambda x, percent: percent if (x >= min_price + tmp_price*1.0*(percent - 1)/10) and (x < min_price + tmp_price*1.0*percent/10) else None


In [40]:
#Moi CATE 1000
#Moi cate 100
# data_final: all category have less feature
data_final = data.icol([0, 1, 2, 3, 4])
data_final.columns = ['Ma_san_pham', 'Category','Ten_san_pham', 'Hang_san_xuat','Gia']
data_final = data_final[~data_final.Category.isin(['42','44', '2002','1942','1943','1944','1962','1992', '1988', '382', '55', '54', '57', '58'])]
#data.Category[data.Category == 4444] = 2
# data_final['CATE2'] = -1
data_final['Category2'] = -2
data_final.Gia = data_final.Gia.apply(lambda x: np.float(x))
data_final['Gia2'] = data_final['Gia']
# CATES = data_final['CATE'].unique().tolist()
# for CATE in CATES:
#     data_final.CATE2[data_final.CATE == CATE] = int(CATES.index(CATE) *100000)
Cates = data_final.Category.unique().tolist()#
for Cate in Cates:
    data_final.Category2[data_final.Category == Cate] = int(Cates.index(Cate) * 100)
    max_price = data_final.Gia[data_final.Category == Cate].max()
    min_price = data_final.Gia[data_final.Category == Cate].min()
    tmp_price = (max_price - min_price)/10

    data_final.Gia2[data_final.Category == Cate] = data_final.Gia2[data_final.Category == Cate].apply(lambda var: rule_gia_all(var))
    Hang_tmp = data_final.Hang_san_xuat[data_final.Category== Cate].tolist()
    data_final.Hang_san_xuat[data_final.Category== Cate] = data_final.Hang_san_xuat[data_final.Category== Cate].apply(lambda x: Hang_tmp.index(x)) + int(Cates.index(Cate) * 100)
       

In [6]:
#data_final.Gia2[data_final.Category == '42'].apply(lambda var: rule_gia_all(var))
#data_final.loc[8288]
#df.to_csv('/home/tungrd/TA/recommenderr/data_maylanh.csv', encoding='utf-8')
# Hang_san_xuat = df['Hang san xuat'].unique()
# Hang_san_xuat
# Noi_san_xuat = df['Noi san xuat'].unique()
# Noi_san_xuat
# Gia = df['Gia'].unique()
# Gia.sort()
#Gia
#check data null
#data_file.isnull().values.any()
#data_file.notnull()

# Cong_suat =df['Cong suat lam lanh'].unique()
# Cong_suat

# Pham_vi = df['Pham vi lam lanh hieu qua'].unique()
# Pham_vi

#df[df['Pham vi lam lanh hieu qua'] == 'Duoi 45 m3']

#dataframe.loc[:'85161']

# recommended by price

In [28]:
# recommended by only price
df = data[data['Category'] == '42']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

In [29]:
#return list of product have the Price difference less than #thread_Price
def Gia_dth(product_id, numofpro, thread_Price, df):
    df.Gia = df.Gia.map(lambda x: float(x))
    df = df.sort(columns=['Gia'])
    return [item for item in df['Ma san pham'].values.tolist() if abs(float(df[df['Ma san pham']==item]['Gia']) - float(df[df['Ma san pham']==product_id]['Gia'])) < thread_Price][:numofpro]


In [30]:
list_price = Gia_dth('88851', -1, 100000, df)
df_name = df[['Ma san pham', 'Ten san pham', 'Hang san xuat', 'Gia']][data['Ma san pham'].isin(list_price)]

In [31]:
df_name

,Ma san pham,Ten san pham,Hang san xuat,Gia
3319,91920,Epic X9 Plus,Epic,3490000.0
3333,88851,Mobiistar Prime X1,Mobiistar,3490000.0
3599,90719,"Asus Zenfone 2 Laser 6"" ZE601KL",Asus,3410000.0
5014,71974,Asus Zenfone Selfie,Asus,3490000.0
9519,70702,Asus Zenfone 2 2.3Ghz/4G/32G,Asus,3490000.0
9600,73039,Lenovo Vibe S1,Lenovo,3490000.0


# Rule dien thoai

In [44]:
#set rule dien thoai

df = data[data['Category'] == '42']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)


def rule_gia_dth(dataframe):
    var = int(dataframe['Gia'])
    if var <= 500000:
        val = 1
    elif var < 1000000:
        val = 2    
    elif var < 1500000:
        val = 3
    elif var < 2000000:
        val = 4
    elif var < 3000000:
        val = 5
    elif var < 5000000:
        val = 6
    elif var < 6000000:
        val = 7     
    elif var < 7000000:
        val = 8
    elif var < 8000000:
        val = 9 
    elif var < 9000000:
        val = 10
    elif var < 10000000:   
        val = 11
    elif var < 11000000:
        val = 12
    elif var < 12000000:   
        val = 13    
    elif var < 13000000:
        val = 14
    elif var < 15000000:   
        val = 15
    elif var < 17000000:   
        val = 16
    elif var >= 17000000:   
        val = 17
    else:
        val = 0 #var > 15tr
    return val + val*2

def rule_hang_dth(dataframe):
    var = (dataframe['Hang san xuat'])
    if var in ['Microsoft','Apple', 'Samsung','Sony','HTC','OPPO','BlackBerry', 'LG']:
        val = 1
    elif var in ['Lenovo', 'Mobiistar', 'Asus', 'Wing', 'Mobell', 'Pantech', 'Oukitel', 'Q-Mobile', 'Coolpad',
 'Xiaomi', 'Intex', 'Itel', 'Philips', 'Bavapen', 'Gionee', 'Freetel', 'Huawei', 'Obi Worldphone', 'Epic']:
        val = 2
   
    else:#Samsung
        val = 2
    return val + val*3

def rule_hang_dth2(dataframe):
    var = (dataframe['Hang san xuat'])
    if var == 'Apple':
        val = a * 5.7
    elif var == 'Samsung':
        val =  a * 6.3   
    elif var =='OPPO':
        val = 11
    elif var =='Sony':
        val = 10
    elif var == 'Microsoft':
        val = a* 6.2
    elif var == 'LG':
        val = 8
    elif var == 'HTC':
        val = 7     
    elif var == 'BlackBerry':
        val = 6
    elif var == 'Xiaomi':
        val = 3.5 
    elif var == 'Lenovo':
        val = 3
    elif var =='Asus':   
        val = 2.5
    elif var =='Huawei':
        val = 2
    elif var in ['Mobiistar', 'Wing', 'Mobell', 'Pantech', 'Oukitel', 'Q-Mobile', 'Coolpad',
 'Intex', 'Itel', 'Philips', 'Bavapen', 'Gionee', 'Freetel', 'Obi Worldphone', 'Epic']:
        val = 1
    else:
        val = 0 #var > 15tr
    return val + val*3

def rule_thoigiansudungpin_dth(dataframe):
    var = (dataframe['Thoi gian su dung pin'])
    if 'Khoang 1 ngay' in var:
        val = 1
    elif 'Khoang 2 ngay' in var:
        val = 2
    elif 'Tren 2 ngay' in var:
        val = 3
    else:# chua cap nhat
        val = 1
    return val + val * .1

def rule_2sim_dth(dataframe):
    var = (dataframe['2 SIM'])#duplicate columns    '2 SIM.1'
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2
    else:# Chua xac dinh
        val = 1
    return val

def rule_camerasau_dth(dataframe):
    var = (dataframe['Camera Sau'])
    if 'Duoi 3 MP' in var:
        val = 1
    elif 'Tu 3 den 5 MP' in var:
        val = 2
    elif 'Tu 5 den 8 MP' in var:
        val = 3
    elif 'Tu 8 den 12 MP' in var:
        val = 4
    elif 'Tren 12 MP' in var:
        val = 5      
    else:# khong camera
        val = 0
    return val

def rule_cameratruoc_dth(dataframe):
    var = (dataframe['Camera Truoc'])
    if 'Duoi 3 MP' in var:
        val = 1
    elif 'Tu 3 den 5 MP' in var:
        val = 2
    elif 'Tu 5 den 8 MP' in var:
        val = 3
    elif 'Tren 8 MP' in var:
        val = 4    
    else:# khong camera
        val = 0
    return val

def rule_thietke_dth(dataframe):
    var = (dataframe['Thiet ke'])
    if var in ['Nguyen khoi', 'Pin lien']:
        val = 2
    elif var == 'Pin roi':
        val = 1
    else:
        val = 1
    return val

def rule_amthanhantuong_dth(dataframe):
    var = (dataframe['Am thanh an tuong'])
    if var == 'Khong':
        val = 1
    elif var == 'Co':
        val = 2
    else:
        val = 1
    return val - val*.2

def rule_RAM_dth(dataframe):
    var = (dataframe['RAM'])#RAM.1
    if var == '512 MB' or var == '768 MB':
        val = 1
    elif var == '1 GB' or var == '1.5 GB':
        val = 2
    elif var == '2 GB':
        val = 3    
    elif var == '3 GB':
        val = 4
    elif var == '4 GB':
        val = 5
    elif var == '6 GB' or var == '5 GB':
        val = 6
    else:
        val = 0
    return val

def rule_bonhotrong_dth(dataframe):
    var = (dataframe['Bo nho trong'])#.1
    if var == 'Khong' or var == 'Chua xac dinh':
        val = 1
    elif var == '4 GB' or var == '8 GB':
        val = 2
    elif var == '16 GB':
        val = 3    
    elif var == '32 GB':
        val = 4
    elif var == '64 GB':
        val = 5
    elif var == '128 GB':
        val = 6
    elif var == '256 GB':
        val = 7   
    elif var == '512 GB':
        val = 8
    elif var == '1 TB':
        val = 9 
    else:
        val = 0
    return val

def rule_dophangiai_dth(dataframe):
    var = (dataframe['Do phan giai.3'])
    if var in ['120 x 160', '128 x 128', '320 x 240']:
        val = 1
    elif var in ['480 x 854', '540 x 960', '480 x 800','640 x 1136']:
        val = 2
    elif var in ['720 x 1280','1334 x 750' ]:
        val = 3    
    elif var in ['1080 x 1920', '1440 x 2560', '1440 x 1440']:
        val = 4 
    else:
        val = 0# Chua xac dinh
    return val

def rule_dorongmanhinh_dth(dataframe):
    var = (dataframe['Man hinh rong'])
    if var in ['1.4"', '1.5"', '1.7"', '1.77"', '1.8"']:
        val = 1
    elif var in ['2.4"',  '2.6"', '2.8"']:
        val = 2
    elif var in [ '4"', '4.5"', '4.7"', '4.8"']:
        val = 3    
    elif var in ['5"', '5.1"','5.15"', '5.2"', '5.3"', '5.5"']:
        val = 4 
    elif var in [ '5.7"',  '6"']:
        val = 5
    else:
        val = 0# Chua xac dinh
    return val + val*.1

def rule_selfie_dth(dataframe):
    var = (dataframe['Selfie dep'])
    if var == 'Khong':
        val = 1
    elif var == 'Co':
        val = 2
    else:
        val = 1
    return val

def rule_hedieuhanh_dth(dataframe):
    var = (dataframe['He dieu hanh'])#.2
    if 'BlackBerry' in var:
        val = 1
    elif 'Windows' in var:
        val = 2
    elif 'iOS' in var:
        val = 3
    elif 'Android' in var:
        val = 4   
    else:
        val = 0
    return val + val*.2

def rule_tocdoCPU_dth(dataframe):
    var = (dataframe['Toc do CPU'])# .1
    if var in ['1 GHz',  '1.1 GHz','1.15 GHz', '1.2 GHz', '1.25 GHz', '1.3 GHz', '1.4 GHz', '1.5 GHz', '1.6 GHz', '1.7 GHz', '1.8 GHz', '1.9 GHz']:
        val = 1
    elif var in ['2.0 GHz', '2.2 GHz', '2.26 GHz', '2.3 GHz', '2.5 GHz'] or '2 nhan' in var:
        val = 2
    elif '4 nhan 1.' in var:
        val = 3    
    elif '4 nhan 2.' in var:
        val = 4 
    elif 'Quad' in var:
        val = 5
    else:
        val = 0# Chua xac dinh
    return val + val*.1

def rule_GPS_dth(dataframe):
    var = (dataframe['GPS'])#.1
    if var == 'Khong':
        val = 0
    elif var == 'Co':
        val = 1
    elif var in ['GLONASS', 'A-GPS']:
        val = 2  
    else:
        val = 0
    return val

df['Gia2'] = df.apply(rule_gia_dth, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_dth, axis = 1)
df['Thoi gian su dung pin2'] = df.apply(rule_thoigiansudungpin_dth, axis = 1)
df['2 SIM2'] = df.apply(rule_2sim_dth, axis=1)
df['Thiet ke2'] = df.apply(rule_thietke_dth, axis = 1)

df['Am thanh an tuong2'] = df.apply(rule_amthanhantuong_dth, axis = 1)
df['RAM2'] = df.apply(rule_RAM_dth, axis = 1)
df['Bo nho trong2'] = df.apply(rule_bonhotrong_dth, axis = 1)
df['Do phan giai2'] = df.apply(rule_bonhotrong_dth, axis = 1)
df['Man hinh rong2'] = df.apply(rule_dorongmanhinh_dth, axis = 1)
df['Selfie dep2'] = df.apply(rule_selfie_dth, axis = 1)
df['He dieu hanh2'] = df.apply(rule_hedieuhanh_dth, axis = 1)
df['Toc do CPU2'] = df.apply( rule_tocdoCPU_dth, axis = 1)
df['GPS2'] = df.apply(rule_GPS_dth, axis = 1)

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Thoi gian su dung pin2', '2 SIM2', 'Thiet ke2',
          'Am thanh an tuong2', 'RAM2','Bo nho trong2',
          'Do phan giai2', 'Man hinh rong2','Selfie dep2' ,'He dieu hanh2','Toc do CPU2','GPS2']]

dataframe_42= calculate_sim(df2)

# Rule may lanh

In [50]:

#set rule may lanh 
df = data[data['Category'] == '2002']
df = df.dropna(axis = 1, how = 'all')

df = df.dropna(axis = 0, how = 'all')
def rule_gia_ml(dataframe):
    var = int(dataframe['Gia'])
    if var < 7000000:
        val = 1
    elif var < 9000000:
        val = 2
    elif var < 12000000:
        val = 3
    elif var < 15000000:
        val = 4
    else:
        val = 5
    return val + val*0.2

def rule_hang_ml(dataframe):
    var = (dataframe['Hang san xuat'])
    if var == 'Daikin':
        val = 1
    elif var == 'Mitsubishi Electric' or var == 'Mitsubishi Electric':
        val = 2
    elif var == 'Panasonic':
        val = 3
    elif var =='Electrolux':
        val = 4
    elif var == 'LG':
        val = 5
    elif var == 'Sanyo':
        val = 6
    elif var =='Toshiba':
        val = 7
    elif var == 'Aqua':
        val = 8
    elif var =='Sharp':
        val = 9        
    else:#Samsung
        val = 10
    return val-val *0.2

def rule_congsuat_ml(dataframe):
    var = (dataframe['Cong suat lam lanh'])
    if '0.75 HP' in var:
        val = 1
    elif '1 HP' in var:
        val = 2
    elif '1.5 HP' in var:
        val = 3
    elif '2 HP' in var:
        val = 4
    else:# 2.5 HP
        val = 5
    return val + val*.1

def rule_loaimay_ml(dataframe):
    var = (dataframe['Loai may.1'])
    if '1 chieu' in var:
        val = 1
    elif '2 chieu' in var:
        val = 2
    elif 'Inverter' in var:
        val = 3  
    else:# May thuong
        val = 0
    return val

def rule_phamvi_ml(dataframe):
    var = (dataframe['Pham vi lam lanh hieu qua'])
    if 'Duoi 15 m2' in var or 'Duoi 45 m3' in var:
        val = 1
    elif 'Tu 15 - 20 m2' in var:
        val = 2
    elif 'Tu 20 - 30 m2' in var:
        val = 3
    elif 'Tu 30 - 40 m2' in var:
        val = 4  
    else:# Tu 45 - 67.5 m3
        val = 5
    return val
def rule_namsanxuat_ml(dataframe):
    var = (dataframe['Nam san xuat'])
    if var == 2013:
        val = 1
    elif var == 2014:
        val = 2
    elif var == 2015:
        val = 3
    elif var == 2016:
        val = 4
    elif var == 2017:
        val = 5
    elif var == 2018:
        val = 6
    elif var == 2019:
        val = 7
    else:
        val = 0
    return val

#set rule may lanh
df['Hang san xuat2'] = df.apply(rule_hang_ml, axis = 1)
df['Cong suat lam lanh2'] = df.apply(rule_congsuat_ml, axis = 1)
df['Pham vi2'] = df.apply(rule_phamvi_ml, axis = 1)
df['Gia2'] = df.apply(rule_gia_ml, axis=1)
df['Loai may2'] = df.apply(rule_loaimay_ml, axis = 1)
df['Nam san xuat2'] = df.apply(rule_namsanxuat_ml, axis = 1)

df2 = df[['Ma san pham','Pham vi2', 'Gia2', 'Cong suat lam lanh2', 'Loai may2', 'Hang san xuat2', 'Nam san xuat2']]

dataframe_2002= calculate_sim(df2)

# Rule laptop

In [51]:
#set rule laptop

df = data[data['Category'] == '44']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_lap(dataframe):
    var = int(dataframe['Gia'])
    if var < 8000000:
        val = 1
    elif var < 10000000:
        val = 2
    elif var < 12000000:
        val = 3
    elif var < 15000000:
        val = 4
    elif var < 20000000:
        val = 5
    else:
        val = 6 #var > 20tr
    return val + val*.2

def rule_hang_lap(dataframe):
    var = (dataframe['Hang san xuat'])
    if 'Apple' in var:
        val = 2
    elif var in ['HP',  'Asus', 'Dell', 'Acer', 'Lenovo', 'Samsung']:
        val = 1
    else:#Samsung
        val = 0
    return val + val*0.3

def rule_trongluong_lap(dataframe):
    var = (dataframe['Trong luong (kg)'])#.1
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 1:
        val = 1
    elif var <= 1.25:
        val = 2
    elif var <= 1.5:
        val = 3
    elif var <= 2:
        val = 4
    elif var <= 2.5:
        val = 5
    elif var <=3:
        val = 6 
    else:# chua cap nhat
        val = 0
    return val

def rule_nhcSudung_lap(dataframe):
    var = (dataframe['Nhu cau su dung'])#duplicate columns
    if 'Hoc tap' in var or 'Van phong' in var:
        val = 1
    elif 'Do hoa' in var:
        val = 2
    elif 'Doanh nhan' in var:
        val = 3
    else:# Chua xac dinh
        val = 1
    return val

def rule_carddohoa_lap(dataframe):
    var = (dataframe['Card do hoa roi'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2    
    else:# khong camera
        val = 1
    return val

def rule_chatlieuvo_lap(dataframe):
    var = (dataframe['Chat lieu vo.1'])#
    if 'Vo nhua' in var:
        val = 1
    elif 'Nhua va Kim loai' in var:
        val = 2
    elif 'Vo kim loai' in var:
        val = 3 
    else:# 
        val = 0
    return val

def rule_kichthuocmanhinh_lap(dataframe):
    var = (dataframe['Kich thuoc man hinh'])#.1
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    if var < 9:
        val = 0
    elif var <=12:
        val = 1
    elif var <=14:
        val = 2
    elif var <= 16:
        val = 3
    else: 
        val =0
    return val + val*0.1

#[u'eMMC', u'HDD', u'HDD + SSD', u'SSHD', u'SSD']
def rule_loaiodia_lap(dataframe):
    var = (dataframe['Loai o dia'])
    if 'HDD' in var:
        val = 1
    elif 'HDD + SSD' in var or 'SSHD' in var:
        val = 2
    elif 'eMMC' in var:
        val = 3
    elif 'SSD' in var:
        val = 4
    else: 
        val = 1
    return val

def rule_RAM_lap(dataframe):
    var = (dataframe['RAM.1'])#.1

    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    if var == 2:
        val = 1
    elif var == 4:
        val = 2
    elif var == 8:
        val = 3    
    elif var == 16:
        val = 4
    else:
        val = 0
    return val


def rule_dophangiai_lap(dataframe):
    var = (dataframe['Do phan giai (W x H)'])
    if 'HD' in var:
        val = 1
    elif 'WXGA' in var:
        val = 2
    elif 'FHD' in var:
        val = 3
    elif 'QHD' in var:
        val = 4
    elif 'Retina' in var:
        val = 5
    else: 
        val = 1
    return val
    
#[u'Atom', u'Celeron', u'Pentium', u'Core i3', u'Core i5', u'Core i7', u'Core M']
def rule_loaiCPU_lap(dataframe):
    var = (dataframe['CPU'])#.1
    if 'Atom' in var:
        val = 1
    elif 'Celeron' in var:
        val = 2
    elif 'Pentium' in var:
        val = 3    
    elif 'Core i3' in var:
        val = 4
    elif 'Core i5' in var:
        val = 5
    elif 'Core i7' in var:
        val = 6
    elif 'Core M' in var:
        val = 7   
    else:
        val = 0
    return val

def rule_todoCPU_lap(dataframe):
    var = (dataframe['Toc do CPU.1'])#.1
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    if var <= 1.2:
        val = 1
    elif var <= 1.6:
        val = 2
    elif var <= 2.2:
        val = 3    
    elif var <= 2.7:
        val = 4
    elif var <= 3.4:
        val = 5    
    else:
        val = 0# Chua xac dinh
    return val

def rule_camung_lap(dataframe):
    var = (dataframe['Man hinh cam ung'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2
    else:
        val = 1# Chua xac dinh
    return val

def rule_thoigianPIN_lap(dataframe):
    var = (dataframe['Thoi gian su dung'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    if var <= 2:
        val = 1
    elif var <=3:
        val = 2
    elif var <=4:
        val = 3
    else:
        val = 1
    return val

def rule_loaiRAM_lap(dataframe):
    var = (dataframe['Loai RAM'])
    if 'DDR3' in var:
        val = 1
    elif 'DDR4' in var:
        val = 2   
    else:
        val = 1
    return val

df['Gia2'] = df.apply(rule_gia_lap, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_lap, axis = 1)
df['Trong luong2'] = df.apply(rule_trongluong_lap, axis = 1)
df['Nhu cau su dung2'] = df.apply(rule_nhcSudung_lap, axis=1)
df['Card do ho roi2'] = df.apply(rule_carddohoa_lap, axis = 1)

df['Chat lieu vo2'] = df.apply(rule_chatlieuvo_lap, axis = 1)
df['Kich thuoc man hinh2'] = df.apply(rule_kichthuocmanhinh_lap, axis = 1)
df['Loai o dia2'] = df.apply(rule_loaiodia_lap, axis = 1)
df['RAM2'] = df.apply(rule_RAM_lap, axis = 1)
df['Do phan giai2'] = df.apply(rule_dophangiai_lap, axis = 1)
df['Loai CPU2'] = df.apply(rule_loaiCPU_lap, axis = 1)
df['Toc do CPU2'] = df.apply(rule_todoCPU_lap, axis = 1)
df['Cam ung2'] = df.apply( rule_camung_lap, axis = 1)
df['Thoi gian PIN2'] = df.apply(rule_thoigianPIN_lap, axis = 1)
df['Loai RAM2'] = df.apply(rule_loaiRAM_lap, axis = 1)

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Trong luong2', 'Nhu cau su dung2', 'Card do ho roi2',
          'Chat lieu vo2', 'Kich thuoc man hinh2','Loai o dia2',
          'RAM2', 'Do phan giai2','Loai CPU2' ,'Toc do CPU2','Cam ung2','Thoi gian PIN2', 'Loai RAM2']]

dataframe_44= calculate_sim(df2)

In [52]:
# concat two dataframe ()
# db_test = pd.concat([df2, db_test], axis= 1)

# Rule Tivi

In [53]:
#### set rule Tivi

df = data[data['Category'] == '1942']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_tv(dataframe):
    var = float(dataframe['Gia'])
    if var < 5000000:
        val = 1
    elif var < 7000000:
        val = 2
    elif var < 10000000:
        val = 3
    elif var < 15000000:
        val = 4
    elif var < 20000000:
        val = 5
    elif var < 50000000:
        val = 6
    elif var < 100000000:
        val = 7    
    elif var < 200000000:
        val = 8
    else:
        val = 0 #var > 20tr
    return val + val*.2

# [u'Asanzo', u'Darling', u'LG', u'Panasonic', u'Samsung', u'Sharp',
#        u'Skyworth', u'Sony', u'TCL', u'Toshiba']
def rule_hang_tv(dataframe):
    var = (dataframe['Hang san xuat'])
    if var in ['Asanzo', 'Darling', 'Skyworth', 'TCL']:
        val = 1
    elif var in ['LG', 'Panasonic', 'Samsung', 'Sharp', 'Sony', 'Toshiba']:
        val = 2
    else:#Samsung
        val = 1
    return val + val*0.1

def rule_congsuatloa_tv(dataframe):
    var = (dataframe['Tong cong suat loa'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 12:
        val = 1
    elif var <= 20:
        val = 2
    elif var <= 30:
        val = 3
    elif var <= 40:
        val = 4
    elif var <= 60:
        val = 5
    elif var <= 80:
        val = 6 
    else:# chua cap nhat
        val = 0
    return val

# [u'Full HD', u'HD', u'Ultra HD 4K']
def rule_dophangian_tv(dataframe):
    var = (dataframe['Do phan giai.4'])#duplicate columns
    if 'HD' in var:
        val = 1
    elif 'Full HD' in var:
        val = 2
    elif 'Ultra HD 4K' in var:
        val = 3
    else:# Chua xac dinh
        val = 0
    return val

def rule_namsanxuat_tv(dataframe):
    var = (dataframe['Nam san xuat.4'])
    if '2015' in var:
        val = 1
    elif '2016' in var:
        val = 2    
    elif '2017' in var:
        val = 3 
    elif '2018' in var:
        val = 4     
    else:# khong camera
        val = 0
    return val

def rule_loaitivi_tv(dataframe):
    var = (dataframe['Loai Tivi.1'])
    if var in ['Tivi LED thuong']:
        val = 1
    elif var in [u'Smart Tivi', u'Internet Tivi',u'Android Tivi']:
        val = 2
    elif var in ['Smart Tivi Cong HDR', 'Android Tivi cong', 'Smart Tivi OLED Cong']:
        val = 3
    elif var in ['Smart Tivi Cong 3D', 'Smart Tivi 3D']:
        val = 4 
    else:# 
        val = 0
    return val - val*0.2

# =============================================================================
df['Gia2'] = df.apply(rule_gia_tv, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_tv, axis = 1)
df['Cong suat loa2'] = df.apply(rule_congsuatloa_tv, axis = 1)
df['Do phan giai2'] = df.apply(rule_dophangian_tv, axis=1)
df['Nam san xuat2'] = df.apply(rule_namsanxuat_tv, axis = 1)

df['Loai tivi2'] = df.apply(rule_loaitivi_tv, axis = 1)

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Cong suat loa2', 'Do phan giai2', 'Nam san xuat2',
          'Loai tivi2']]

dataframe_1942= calculate_sim(df2)

# Rule tủ lạnh

In [54]:
#### set rule Tủ lạnh
df = data[data['Category'] == '1943']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

# df['Gia'].apply(lambda x: int(x)).sort_values(inplace = False, ascending = True).unique()
# df['Dung tich tren tem nang luong'].unique()

def rule_gia_tl(dataframe):
    var = float(dataframe['Gia'])
    if var < 5000000:
        val = 1
    elif var < 7000000:
        val = 2
    
    elif var < 10000000:
        val = 3
    elif var < 15000000:
        val = 4
    elif var < 20000000:
        val = 5
    elif var < 30000000:
        val = 6
    elif var < 40000000:
        val = 7    
    elif var < 60000000:
        val = 8
    elif var < 100000000:
        val = 9
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_hang_tl(dataframe):
    var = (dataframe['Hang san xuat'])
    if var in ['Darling', 'Midea', 'Beko']:
        val = 1
    elif var in ['Aqua' , 'Panasonic', 'Sharp', 'LG',
        'Mitsubishi Electric', 'Sanyo', 'Toshiba', 'Samsung',
        'Electrolux', 'Hitachi']:
        val = 2
    else:#Samsung
        val = 1
    return val 

def rule_dungtichtong_tl(dataframe):
    var = (dataframe['Dung tich tren tem nang luong'])

    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 100:
        val = 1
    elif var <= 150:
        val = 2
    elif var <= 200:
        val = 3
    elif var <= 250:
        val = 4
    elif var <= 300:
        val = 5
    elif var <= 400:
        val = 6 
    elif var <= 500:
        val = 7 
    elif var <= 600:
        val = 8
    elif var >600:
        val = 9
    else:# chua cap nhat
        val = 0
    return val + val*0.1

# [u'Full HD', u'HD', u'Ultra HD 4K']
def rule_inverter_tl(dataframe):
    var = (dataframe['Cong nghe Inverter.1'])#duplicate columns
    if 'Tu lanh thuong' in var:
        val = 1
    elif 'Tu lanh Inverter' in var:
        val = 2
    else:# Chua xac dinh
        val = 1
    return val

def rule_kieutu_tl(dataframe):
    var = (dataframe['Kieu tu'])
    if 'Mini' in var:
        val = 1
    elif 'Ngan da tren' in var:
        val = 2    
    elif 'Ngan da duoi' in var:
        val = 3 
    elif 'Door in door' in var or 'Side by side' in var:
        val = 4     
    else:# khong camera
        val = 0
    return val

def rule_namsanxuat_tl(dataframe):
    var = (dataframe['Nam san xuat.3'])
    if '2013' in var:
        val = 1
    elif '2014' in var:
        val = 2    
    elif '2015' in var:
        val = 3 
    elif '2016' in var:
        val = 4     
    elif '2017' in var:
        val = 5 
    elif '2018' in var:
        val = 6
    elif '2019' in var:
        val = 7 
    elif '2020' in var:
        val = 8    
    else:# khong camera
        val = 0
    return val

def rule_socua_tl(dataframe):
    var = (dataframe['So cua'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 1:
        val = 1
    elif var <= 2:
        val = 2
    elif var <= 3:
        val = 3
    elif var <= 4:
        val = 4
    elif var <= 5:
        val = 5
    elif var <= 6:
        val = 6 
    else:# chua cap nhat
        val = 0
    return val - val*0.1

# =============================================================================
df['Gia2'] = df.apply(rule_gia_tl, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_tl, axis = 1)
df['Dung tich tong2'] = df.apply(rule_dungtichtong_tl, axis = 1)
df['Inverter2'] = df.apply(rule_inverter_tl, axis=1)
df['Nam san xuat2'] = df.apply(rule_namsanxuat_tl, axis = 1)
df['Kieu tu2'] = df.apply(rule_kieutu_tl, axis = 1)
df['So cua2'] = df.apply(rule_socua_tl, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Dung tich tong2', 'Inverter2',
          'Nam san xuat2', 'So cua2']]

dataframe_1943= calculate_sim(df2)

# Rule máy giặt

In [55]:
# set rule Máy giặt
df = data[data['Category'] == '1944']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

# df['Gia'].apply(lambda x: int(x)).sort_values(inplace = False, ascending = True).unique()
# df['Cong nghe giat.1'].unique()

def rule_gia_mg(dataframe):
    var = float(dataframe['Gia'])
    if var < 4000000:
        val = 1
    elif var < 5000000:
        val = 2
    elif var < 70000000:
        val = 3
    elif var < 9000000:
        val = 4
    elif var < 120000000:
        val = 5
    elif var < 15000000:
        val = 6
    elif var < 20000000:
        val = 7    
    elif var < 30000000:
        val = 8
    elif var < 500000000:
        val = 9
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_hang_mg(dataframe):
    var = (dataframe['Hang san xuat'])
    if var in ['Midea', 'Beko']:
        val = 1
    elif var in ['Aqua' , 'Panasonic', 'Sharp', 'LG',
          'Toshiba', 'Samsung',
        'Electrolux', 'Hitachi']:
        val = 2
    else:#Samsung
        val = 1
    return val 

def rule_khoiluonggiat_mg(dataframe):
    var = (dataframe['Khoi luong giat.2'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 6:
        val = 1
    elif var <= 7:
        val = 2
    elif var <= 8:
        val = 3
    elif var <= 9:
        val = 4
    elif var <= 10:
        val = 5
    elif var <= 11:
        val = 6 
    elif var <= 13:
        val = 7 
    elif var <= 15:
        val = 8
    elif var >15:
        val = 9
    else:# chua cap nhat
        val = 0
    return val + val*0.1

def rule_loaimaygiat_mg(dataframe):
    var = (dataframe['Loai may giat'])
    if 'Cua tren' in var:
        val = 1
    elif 'May giat say cua truoc' in var:
        val = 3    
    elif 'Cua truoc' in var:
        val = 2   
    else:# khong camera
        val = 0
    return val

def rule_longgiat_mg(dataframe):
    var = (dataframe['Long giat'])
    if 'Long dung' in var:
        val = 1
    elif 'Long nghieng' in var:
        val = 2    
    elif 'Long ngang' in var:
        val = 3   
    else:# khong camera
        val = 0
    return val
# 
def rule_namsanxuat_mg(dataframe):
    var = (dataframe['Nam san xuat.2'])
    if '2013' in var:
        val = 1
    elif '2014' in var:
        val = 2    
    elif '2015' in var:
        val = 3 
    elif '2016' in var:
        val = 4     
    elif '2017' in var:
        val = 5 
    elif '2018' in var:
        val = 6
    elif '2019' in var:
        val = 7 
    elif '2020' in var:
        val = 8    
    else:# khong camera
        val = 0
    return val

def rule_inverter_mg(dataframe):
    var = (dataframe['Inverter'])
    
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2
    else:# chua cap nhat
        val = 0
    return val - val*0.1

# [u'Giat co dac bang bot khi', u'Long giat kim cuong the he 2']
def rule_congnghegiat_mg(dataframe):
    var = (dataframe['Cong nghe giat.1'])
    
    if 'Giat co dac bang bot khi' in var:
        val = 1
    elif 'Long giat kim cuong the he 2' in var:
        val = 2
    else:# chua cap nhat
        val = 0
    return val - val*0.1

# =============================================================================
df['Gia2'] = df.apply(rule_gia_mg, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_mg, axis = 1)
df['Khoi luong giat2'] = df.apply(rule_khoiluonggiat_mg, axis = 1)
df['Inverter2'] = df.apply(rule_inverter_mg, axis=1)
df['Nam san xuat2'] = df.apply(rule_namsanxuat_mg, axis = 1)
df['Loai may giat2'] = df.apply(rule_loaimaygiat_mg, axis = 1)
df['Long giat2'] = df.apply(rule_longgiat_mg, axis = 1)
df['Cong nghe giat2'] = df.apply(rule_congnghegiat_mg, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Khoi luong giat2', 'Inverter2', 'Nam san xuat2',
          'Loai may giat2', 'Long giat2', 'Cong nghe giat2']]
dataframe_1944= calculate_sim(df2)

# Rule Máy nước nóng

In [56]:
#### set rule Máy nước nóng
df = data[data['Category'] == '1962']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

# df['Gia'].apply(lambda x: int(x)).sort_values(inplace = False, ascending = True).unique()
# df['Chong khuan voi v'].unique()

def rule_gia_mnn(dataframe):
    var = float(dataframe['Gia'])
    if var < 2000000:
        val = 1
    elif var < 2500000:
        val = 2
    elif var < 3000000:
        val = 3
    elif var < 4000000:
        val = 4
    elif var < 5000000:
        val = 5
    elif var <= 6000000:
        val = 6
    else:
        val = 0 #var > 20tr
    return val + val*.2

# [u'Ferroli', u'Electrolux', u'Ariston', u'Panasonic', u'Centon',
#        u'Atlantic', u'Hitachi']
def rule_hang_mnn(dataframe):
    var = (dataframe['Hang san xuat'])
    if 'Ferroli' in var:
        val = 1
    elif 'Electrolux' in var:
        val = 2
    elif 'Ariston' in var:
        val = 3
    elif 'Panasonic' in var:
        val = 4
    elif 'Centon' in var:
        val = 5
    elif 'Atlantic' in var:
        val = 6 
    elif 'Hitachi' in var:
        val = 7
    else:#Samsung
        val = 0
    return val 

# [u'4500 W', u'2500 W', u'3500 W', u'6000 W', u'1500 W + 1500 W',
#        u'nan', u'4400 W']
def rule_congsuatlamnong_mnn(dataframe):
    var = (dataframe['Cong suat lam nong'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 1500:
        val = 1
    elif var <= 2500:
        val = 2
    elif var <= 3500:
        val = 3
    elif var <= 4500:
        val = 4
    elif var <= 6000:
        val = 5
    else:# chua cap nhat
        val = 0
    return val + val*0.1

def rule_dungtich_mnn(dataframe):
    var = (dataframe['Dung tich binh chua'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 15:
        val = 1
    elif var <= 20:
        val = 2
    elif var <= 30:
        val = 3
    else:# chua cap nhat
        val = 0
    return val

# [u'Lam nong truc tiep', u'Lam nong gian tiep']
def rule_loaimay_mnn(dataframe):
    var = (dataframe['Loai may.2'])
    if 'truc tiep' in var:
        val = 1
    elif 'gian tiep' in var:
        val = 2      
    else:# khong camera
        val = 0
    return val

def rule_chongkhuan_mnn(dataframe):
    var = (dataframe['Chong khuan voi sen'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2      
    else:# khong camera
        val = 1
    return val

def rule_bomtroluc_mnn(dataframe):
    var = (dataframe['Bom tro luc'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2      
    else:# khong camera
        val = 1
    return val
# 
# [u'Co', u'Khong', u'Bo on nhiet', u'nan']
def rule_cambiennhiet_mnn(dataframe):
    var = (dataframe['Cam bien nhiet']) 
    if var in ['Co', 'Bo on nhiet']:
        val = 2
    elif 'Khong' in var:
        val = 1
    else:# chua cap nhat
        val = 1
    return val - val*0.1

# =============================================================================
df['Gia2'] = df.apply(rule_gia_mnn, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hang_mnn, axis = 1)
df['Cong suat lam nong2'] = df.apply(rule_congsuatlamnong_mnn, axis = 1)
df['Dung tich binh2'] = df.apply(rule_dungtich_mnn, axis=1)
df['Loai may2'] = df.apply(rule_loaimay_mnn, axis = 1)
df['Chong khuan2'] = df.apply(rule_chongkhuan_mnn, axis = 1)
df['Bom tro luc2'] = df.apply(rule_bomtroluc_mnn, axis = 1)
df['Cam bien nhiet2'] = df.apply(rule_cambiennhiet_mnn, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Cong suat lam nong2', 'Dung tich binh2', 'Chong khuan2',
          'Loai may2', 'Bom tro luc2', 'Bom tro luc2']]
dataframe_1962= calculate_sim(df2)

# Rule Quạt máy

In [57]:
#### set rule Quạt máy
df = data[data['Category'] == '1992']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

# df['Gia'].apply(lambda x: int(x)).sort_values(inplace = False, ascending = True).unique()
# df['Chong khuan voi v'].unique()

def rule_gia_qm(dataframe):
    var = float(dataframe['Gia'])
    if var < 200000:
        val = 1
    elif var < 500000:
        val = 2
    elif var < 1000000:
        val = 3
    elif var < 1500000:
        val = 4
    elif var < 2000000:
        val = 5
    elif var <= 3000000:
        val = 6
    elif var <= 4000000:
        val = 7
    elif var <= 6000000:
        val = 8
    elif var > 6000000:
        val = 9
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_hangsx_qm(dataframe):
    var = (dataframe['Hang san xuat'])
    
    if 'Senko' in var:
        val = 0.5
    elif 'Asia' in var:
        val = 1
    elif 'Lifan' in var:
        val = 1.5
    elif 'Midea' in var:
        val = 2
    elif 'Electrolux' in var:
        val = 2.5
    elif 'Sunhouse' in var:
        val = 3 
    elif 'Sanyo' in var:
        val = 3.5
    elif 'Panasonic' in var:
        val = 4
    elif 'Mitsubishi Electric' in var:
        val = 4.5
    elif 'KDK' in var:
        val = 5 
    elif 'Tefal' in var:
        val = 5.5
    elif 'Sanaky' in var:
        val = 6
    elif 'Magic' in var:
        val = 6.5
    elif 'Aqua' in var:
        val = 7
    elif 'Kangaroo' in var:
        val = 7.5
    elif 'Iruka' in var:
        val = 8
    else:#Samsung
        val = 0
    return val 

def rule_loai_qm(dataframe):
    var = (dataframe['Loai quat'])
    if 'Quat ban' in var:
        val = 1
    elif 'Quat hop' in var:
        val = 2
    elif 'Quat lung' in var:
        val = 3
    elif 'Quat treo tuong' in var:
        val = 4
    elif 'Quat dung' in var:
        val = 5
    elif 'Quat sac dien' in var:
        val = 6 
    elif 'Quat phun suong' in var:
        val = 7
    elif 'Quat hoi nuoc' in var:
        val = 7.5
    elif 'Quat dieu hoa' in var:
        val = 8
    else:#Samsung
        val = 0
    return val 

def rule_congsuat_qm(dataframe):
    var = (dataframe['Cong suat.2'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 30:
        val = 1
    elif var <= 40:
        val = 2
    elif var <= 50:
        val = 3
    elif var <= 60:
        val = 4
    elif var <= 80:
        val = 5
    elif var <= 100:
        val = 6
    elif var > 100:
        val = 7
    else:# chua cap nhat
        val = 0
    return val + val*0.1

# =============================================================================
df['Gia2'] = df.apply(rule_gia_qm, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hangsx_qm, axis = 1)
df['Cong suat2'] = df.apply(rule_congsuat_qm, axis = 1)
df['Loai may2'] = df.apply(rule_loai_qm, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Cong suat2', 'Loai may2']]
dataframe_1992= calculate_sim(df2)

# Rule Bàn ủi

In [58]:
#### set rule Bàn ủi
df = data[data['Category'] == '1988']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_bu(dataframe):
    var = float(dataframe['Gia'])
    if var < 200000:
        val = 1
    elif var < 300000:
        val = 2
    elif var < 500000:
        val = 3
    elif var < 700000:
        val = 4
    elif var < 1000000:
        val = 5
    elif var <= 1500000:
        val = 6
    elif var <= 2000000:
        val = 7
    elif var <= 3000000:
        val = 8
    elif var > 3000000:
        val = 9
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_hangsx_bu(dataframe):
    var = (dataframe['Hang san xuat'])
    if var in ['Comet', 'Sunhouse', 'Pensonic']:
        val = 1
    elif var in ['Delites' , u'Bluestone', u'Electrolux', u'Philips', u'Panasonic', u'Tefal', u'Hitachi']:
        val = 2

    return val 

def rule_loai_bu(dataframe):
    var = (dataframe['Loai ban ui'])
    if 'Ban ui kho' in var:
        val = 1
    elif 'Ban ui hoi nuoc' in var:
        val = 2
    elif 'Ban ui hoi nuoc dung' in var:
        val = 3
    else:#Samsung
        val = 0
    return val 

def rule_congsuat_bu(dataframe):
    var = (dataframe['Cong suat.7'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>0:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 1000:
        val = 1
    elif var <= 1400:
        val = 2
    elif var <= 1800:
        val = 3
    elif var <= 2500:
        val = 4
    elif var <= 3000:
        val = 5
    elif var > 3000:
        val = 6
    else:# chua cap nhat
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_bu, axis = 1)
df['Hang san xuat2'] = df.apply(rule_hangsx_bu, axis = 1)
df['Cong suat2'] = df.apply(rule_congsuat_bu, axis = 1)
df['Loai may2'] = df.apply(rule_loai_bu, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2','Hang san xuat2', 'Cong suat2', 'Loai may2']]
dataframe_1998= calculate_sim(df2)

# Rule Loa


In [59]:
#### set rule Loa
df = data[data['Category'] == '382']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_loa(dataframe):
    var = float(dataframe['Gia'])
    if var < 200000:
        val = 1
    elif var < 300000:
        val = 2
    elif var < 500000:
        val = 3
    elif var < 700000:
        val = 4
    elif var < 1000000:
        val = 5
    elif var <= 1500000:
        val = 6
    elif var <= 2000000:
        val = 7
    elif var <= 3000000:
        val = 8
    elif var <= 4000000:
        val = 9
    elif var > 4000000:
        val = 10
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_loai_loa(dataframe):
    var = (dataframe['Chon loai loa'])
    if 'Loa co day' in var:
        val = 1
    elif 'Loa Bluetooth' in var:
        val = 2
    else:#Samsung
        val = 0
    return val * 2

def rule_congsuat_loa(dataframe):
    var = (dataframe['Tong Cong suat (W)'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)==2:
        var = float(var[0]) * float(var[1])
    elif len(var) == 1:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 6:
        val = 1
    elif var <= 12:
        val = 2
    elif var <= 20:
        val = 3
    elif var <= 30:
        val = 4
    elif var <= 40:
        val = 5
    elif var <= 60:
        val = 6
    elif var > 60:
        val = 7
    else:# chua cap nhat
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_loa, axis = 1)
df['Cong suat2'] = df.apply(rule_congsuat_loa, axis = 1)
df['Loai loa2'] = df.apply(rule_loai_loa, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2', 'Cong suat2', 'Loai loa2']]
dataframe_382= calculate_sim(df2)

In [60]:
# concate two dataframe
# db_test = pd.concat([df2, db_test], axis= 1 )

# Rule Thẻ nhớ

In [61]:
#### set rule Thẻ nhớ
df = data[data['Category'] == '55']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_thnh(dataframe):
    var = float(dataframe['Gia'])
    if var < 100000:
        val = 1
    elif var < 200000:
        val = 2
    elif var < 300000:
        val = 3
    elif var < 400000:
        val = 4
    elif var < 500000:
        val = 5
    elif var <= 700000:
        val = 6
    elif var <= 1000000:
        val = 7
    elif var <= 1500000:
        val = 8
    elif var <= 2000000:
        val = 9
    elif var > 3000000:
        val = 10
    elif var <= 4000000:
        val = 11
    elif var > 4000000:
        val = 12
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_loai_thnh(dataframe):
    var = (dataframe['Loai the'])
    if 'The MicroSD' in var:
        val = 1
    elif 'The SD' in var:
        val = 2
    elif 'The CF' in var:
        val = 3
    elif 'Macbook' in var:
        val = 10
    else:#Samsung
        val = 0
    return val

def rule_dungluong_thnh(dataframe):
    var = (dataframe['Dung luong the'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>= 1:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 4:
        val = 1
    elif var <= 8:
        val = 2
    elif var <= 16:
        val = 3
    elif var <= 32:
        val = 4
    elif var <= 64:
        val = 5
    elif var <= 128:
        val = 6
    elif var > 128:
        val = 7
    else:# chua cap nhat
        val = 0
    return val

def rule_tocdo_thnh(dataframe):
    var = (dataframe['Toc do doc/ ghi du lieu'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>= 1:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 10:
        val = 1
    elif var <= 30:
        val = 2
    elif var <= 50:
        val = 3
    elif var < 80:
        val = 4
    elif var <= 100:
        val = 5
    elif var <= 120:
        val = 6
    elif var > 120:
        val = 7
    else:# chua cap nhat
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_thnh, axis = 1)
df['Dung luong the nho2'] = df.apply(rule_dungluong_thnh, axis = 1)
df['Loai the nho2'] = df.apply(rule_loai_thnh, axis = 1)
df['Toc do2'] = df.apply(rule_tocdo_thnh, axis = 1)

df2 = df[['Ma san pham','Gia2', 'Dung luong the nho2', 'Loai the nho2', 'Toc do2']]
dataframe_55 = calculate_sim(df2)

In [62]:
# concat two dataframe
# db_test = pd.concat([df2, db_test], axis= 1 )

# Rule Tai nghe

In [63]:
#### set rule Tai nghe
df = data[data['Category'] == '54']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_tngh(dataframe):
    var = float(dataframe['Gia'])
    if var < 100000:
        val = 1
    elif var < 200000:
        val = 2
    elif var < 300000:
        val = 3
    elif var < 500000:
        val = 4
    elif var < 700000:
        val = 5
    elif var <= 1000000:
        val = 6
    elif var <= 1500000:
        val = 7
    elif var <= 2000000:
        val = 8
    elif var <= 2500000:
        val = 9
    elif var > 3000000:
        val = 10
    elif var <= 4000000:
        val = 11
    elif var > 4000000:
        val = 12
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_loai_tngh(dataframe):
    var = (dataframe['Loai tai nghe'])
    if 'co day' in var:
        val = 1
    elif 'Bluetooth' in var:
        val = 2
    else:#Samsung
        val = 0
    return val

def rule_jackcam_tngh(dataframe):
    var = (dataframe['Jack cam'])
    if '3.5' in var:
        val = 3
    elif '2.5' in var:
        val = 8
    elif 'Micro' in var:
        val = 13
    elif 'Mini' in var:
        val = 8
    else:#Samsung
        val = 0
    return val

def rule_kieu_tngh(dataframe):
    var = (dataframe['Kieu tai nghe'])
    if 'nhet trong' in var:
        val = 1
    elif 'chup tai' in var:
        val = 2
    else:#Samsung
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_tngh, axis = 1)
df['Kieu tai nghe2'] = df.apply(rule_kieu_tngh, axis = 1)
df['Loai tai nghe2'] = df.apply(rule_loai_tngh, axis = 1)
df['Jack2'] = df.apply(rule_jackcam_tngh, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2', 'Kieu tai nghe2', 'Loai tai nghe2', 'Jack2']]
dataframe_54= calculate_sim(df2)

In [64]:
# concat two dataframe
# db_test = pd.concat([df2, db_test], axis= 1 )

# Rule day sac

In [71]:
# df = data[data['Category'] == '58']
# df.columns.unique().tolist()

In [70]:
#### set rule Dây sạc
df = data[data['Category'] == '58']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_sac(dataframe):
    var = float(dataframe['Gia'])
    if var < 100000:
        val = 1
    elif var < 250000:
        val = 2
    elif var < 300000:
        val = 3
    elif var < 400000:
        val = 4
    elif var < 500000:
        val = 5
    elif var <= 700000:
        val = 6
    elif var > 700000:
        val = 12
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_loaisp_sac(dataframe):
    var = (dataframe['Loai san pham.1'])

    if 'Micro USB' in var:
        val = 1
    elif 'Cap chuyen doi'in var:
        val = 2
    elif 'iPhone' in var:
        val = 3
    elif 'Adapter' in var or 'cu sac' in var:
        val = 4
    elif 'Type C'in var:
        val = 5
    elif 'Nokia dau nho' in var:
        val = 6
    else:# chua cap nhat
        val = 0
    return val*10

def rule_jack_sac(dataframe):
    list_jack = ['Micro USB', '3.5mm', 'iPhone (lightning)', 'USB', 'iPhone 4',
       'Ho tro tat ca', 'USB Type-C', 'Nokia dau nho', 'Type-C',
       'Micro USB va Apple lightning', 'Micro USB va iPhone 5', 'Khac',
       'HDMI 1.4', 'Micro USB va Type C', 'HDMI 2.0']
    var = (dataframe['Jack ket noi'])
    if var in list_jack:
        val = list_jack.index(var)
    else:
        val = len(list_jack)
    return val

def rule_dungtrenoto_sac(dataframe):
    var = (dataframe['Dung tren Oto'])
    if 'Khong' in var:
        val = 1
    elif 'USB 2.0' in var:
        val = 2
    elif 'Co' in var:
        val = 3
    else:#Samsung
        val = 0
    return val

def rule_nguonra_sac(dataframe):
    var = (dataframe['Nguon ra'])
    var1 = re.findall(r"\d+\.?\d*A", str(var)) #var = re.findall(r"\d+\.?\d*V|\d+\.?\d*A", str(var))
    if len(var1)== 1:
        tmp =float(re.findall(r"\d+\.?\d*", str(var1[0]))[0])
    elif len(var1) >= 2:    
        tmp = 99
    else:
        tmp = 100
    
    if tmp <= 1:
        val = 1
    elif tmp <= 2:
        val = 2
    elif tmp <=3:
        val = 3
    elif tmp <= 6:
        val = 4
    elif tmp ==99:
        val = 5
    else:# chua cap nhat, tmp = 100
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_sac, axis = 1)
# rule_gia_sac
df['Loai san pham2'] = df.apply(rule_loaisp_sac, axis = 1)
df['Jack sac2'] = df.apply(rule_jack_sac, axis = 1)
df['Dung tren oto2'] = df.apply(rule_dungtrenoto_sac, axis = 1)
df['Nguon ra2'] = df.apply(rule_nguonra_sac, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2', 'Loai san pham2', 'Jack sac2', 'Dung tren oto2', 'Nguon ra2']]
dataframe_58= calculate_sim(df2)

In [ ]:
# concat two dataframe
# db_test = pd.concat([df2, db_test], axis= 1 )

# Rule pin dự phòng

In [72]:
#### set rule Pin dự phòng
df = data[data['Category'] == '57']
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

def rule_gia_pin(dataframe):
    var = float(dataframe['Gia'])
    if var < 100000:
        val = 1
    elif var < 250000:
        val = 2
    elif var < 300000:
        val = 3
    elif var < 400000:
        val = 4
    elif var < 500000:
        val = 5
    elif var <= 700000:
        val = 6
    elif var > 700000:
        val = 12
    else:
        val = 0 #var > 20tr
    return val + val*.2

def rule_dungluong_pin(dataframe):
    var = (dataframe['Dung luong'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>= 1:
        var = float(var[0])
    else:
        var = np.nan
    
    if var <= 5000:
        val = 1
    elif var <= 10000:
        val = 2
    elif var <= 15000:
        val = 3
    elif var <= 20000:
        val = 4
    elif var > 20000:
        val = 5
    else:# chua cap nhat
        val = 0
    return val

# Thoi gian sac
def rule_thoigiansac_pin(dataframe):
    var = (dataframe['Thoi gian sac.1'])
    var = re.findall(r"\d+\.?\d*", str(var))
    if len(var)>= 2:
        var = (float(var[0]) + float(var[1]))/2
    elif len(var)==1:
        var = float(var[0])
    else:
        var = np.nan
    if var <= 2:
        val = 7
    elif var <= 3:
        val = 6
    elif var <= 4:
        val = 5
    elif var <= 5:
        val = 4
    elif var <= 6:
        val = 3
    elif var <= 7:
        val = 2    
    elif var > 7:
        val = 1
    else:# chua cap nhat
        val = 0
    return val

def rule_ngatsac_pin(dataframe):
    var = (dataframe['Tu dong ngat khi sac day dien thoai'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2
    else:#Samsung
        val = 0
    return val

def rule_denled_pin(dataframe):
    var = (dataframe['Den LED bao hieu'])
    if 'Khong' in var:
        val = 1
    elif 'Co' in var:
        val = 2
    else:#Samsung
        val = 0
    return val

# =============================================================================
df['Gia2'] = df.apply(rule_gia_pin, axis = 1)
df['Dung luong pin2'] = df.apply(rule_dungluong_pin, axis = 1)
df['Thoi gian sac2'] = df.apply(rule_thoigiansac_pin, axis = 1)
df['Ngat sac2'] = df.apply(rule_ngatsac_pin, axis = 1)
df['Den led2'] = df.apply(rule_denled_pin, axis = 1)
# rule_kieutu_tl

df2 = df[['Ma san pham','Gia2', 'Dung luong pin2', 'Thoi gian sac2', 'Ngat sac2', 'Den led2']]
dataframe_57= calculate_sim(df2)

In [ ]:
# concat two dataframe
# db_test = pd.concat([df2, db_test], axis= 1 )

In [ ]:
# 
# db_test1 = db_test
# db_test1 = db_test1.groupby(db_test1.columns, axis=1).agg(np.max)

In [ ]:
# the end

# Calculate: one-df2

In [42]:
#calculate the similarity for special (one) CATE
def calculate_sim(df2):
    dataframe = pd.DataFrame(index = df2['Ma san pham'], columns= [np.arange(df2.shape[0])])
    for a in range(df2.shape[0]):
        for b in range(df2.shape[0]):
            dataframe.iloc[a][b] = length2(df2.iloc[a][1:], df2.iloc[b][1:])
    dataframe.columns = df2['Ma san pham']
    
    return dataframe

# Calculate: multi: data_final

In [32]:
# #calculate the similarity all items
# start_time = time.time()
# CATES = data_final['CATE'].unique().tolist()

# #list_shape = []
# for CATE in CATES:
#     df_tmp = data_final[data_final.CATE == CATE]#get sub dataframe
    
#     dataframe_ = pd.DataFrame(index = df_tmp['Ma_san_pham'], columns= [np.arange(df_tmp.shape[0])])
#     dataframe_.columns = df_tmp['Ma_san_pham']
#     #list_shape.append(CATE)
#     #list_shape.append(dataframe_.shape)
#     #asign values to dataframe
#     for a in range(df_tmp.shape[0]):
#         for b in range(df_tmp.shape[0]):
#             dataframe_.iloc[a][b] = length2(df_tmp.iloc[a][[4, 6, 7, 8]], df_tmp.iloc[b][[4, 6, 7, 8]])
#     #Add to redis
#     for x in dataframe_.columns.values.tolist():
#         tmp = list_similar(x, 5, dataframe_)
#         for y in range(5):
#             rc.zadd('RD_'+x,tmp.values[y], tmp.index[y])  
            
# #     #Delete 
# #     for x in dataframe_.columns.values.tolist():
# #         rc.delete(str('RD_'+x)) 
        
# run_time = time.time() - start_time


In [42]:
# #calculate the similarity all items, 
# start_time = time.time()
# CATES = data_final['CATE'].unique().tolist()

# #list_shape = []
# for CATE in CATES:
#     df_tmp = data_final[data_final.CATE == CATE]#get sub dataframe
    
#     exec('dataframe_'+str(CATE) +'= pd.DataFrame(index = df_tmp.Ma_san_pham, columns= [np.arange(df_tmp.shape[0])])')
#     exec('dataframe_'+str(CATE)+'.columns = df_tmp.Ma_san_pham')
#     #list_shape.append(CATE)
#     #list_shape.append(dataframe_.shape)
#     #asign values to dataframe
#     for a in range(df_tmp.shape[0]):
#         for b in range(df_tmp.shape[0]):
#             exec('dataframe_'+str(CATE)+ '.iloc[a][b] = length2(df_tmp.iloc[a][[4, 6, 7, 8]], df_tmp.iloc[b][[4, 6, 7, 8]])')
# #     #Add to redis
# #     for x in dataframe_.columns.values.tolist():
# #         tmp = list_similar(x, 5, dataframe_)
# #         for y in range(5):
# #             rc.zadd('RD_'+x,tmp.values[y], tmp.index[y])  
            
# #     #Delete 
# #     for x in dataframe_.columns.values.tolist():
# #         rc.delete(str('RD_'+x)) 
        
# run_time = time.time() - start_time


In [43]:
#calculate the similarity all items, 
start_time = time.time()
Category = data_final['Category'].unique().tolist()

for CATE in Category:
    df_tmp = data_final[data_final.Category == CATE]#get sub dataframe

    exec('dataframe_'+str(CATE) +'= pd.DataFrame(index = df_tmp.Ma_san_pham, columns= [np.arange(df_tmp.shape[0])])')
    exec('dataframe_'+str(CATE)+'.columns = df_tmp.Ma_san_pham')
    #list_shape.append(CATE)
    #list_shape.append(dataframe_.shape)
    #asign values to dataframe
    for a in range(df_tmp.shape[0]):
        for b in range(df_tmp.shape[0]):
            exec('dataframe_'+str(CATE)+ '.iloc[a][b] = length2(df_tmp.iloc[a][[3, 5, 6]], df_tmp.iloc[b][[3, 5, 6]])')

# #Add to redis
# for x in dataframe_.columns.values.tolist():
#     tmp = list_similar(x, 5, dataframe_)
#     for y in range(5):
#         rc.zadd('RD_'+x,tmp.values[y], tmp.index[y])  

# #Delete 
# for x in dataframe_.columns.values.tolist():
#     rc.delete(str('RD_'+x)) 
        
run_time = time.time() - start_time


SyntaxError: invalid syntax (<string>, line 1)

In [446]:
#check category have how many feature to calculate the similary
list_tmp = []
CATES = data_final['Category'].unique().tolist()

#list_shape = []
for CATE in CATES:
    df = data[data['Category'] == CATE]
    #drop all those columns like (value, value2, value3) where all rows have same values
    cols = list(df)
    nunique = df.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    df = df.drop(cols_to_drop, axis=1)

    list_tmp.append(CATE +': '+ str(df.shape[1]))
    
# output
# list_tmp

In [116]:
# #write to excel
# ew = pd.ExcelWriter('/home/tungrd/TA/recommenderr/thesameall.xlsx', options={'encoding':'utf-8'})
# dataframe.to_excel(ew, 'thesameall')
# ew.save()

# #read file excel
# dataframe = pd.read_excel('/home/tungrd/TA/recommenderr/thesameall.xlsx', sheetname='thesameall', header = 0, index_col=0 )

    # Visualization demo

In [25]:
#list_similar(id_produc, number_of_recommended_product)
a = list_similar('88538', 10, dataframe)
#a= dataframe_['76636'].sort(inplace = False)[:10]

In [3]:
#review (change df2 or data is name of dataframe)
a_value = map(lambda x: np.str(x), list(a.index))
df.loc[data['Ma san pham'].isin(a_value)]
# data_final.loc[data['Ma san pham'].isin(a_value)]

# Save to Redis

In [2]:
# pip install redis-py-cluster
import redis
from rediscluster import StrictRedisCluster
# Requires at least one node for cluster discovery. Multiple nodes is recommended.
startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)
#!redis-cli -h 10.1.12.30 -p 16379 -c
#rc.dbsize()

In [248]:
#Add to redis
for x in dataframe.columns.values.tolist():
    tmp = list_similar(x, 20, dataframe)
    for y in range(20):
        rc.zadd('RD_'+x,tmp.values[y], tmp.index[y])

In [4]:
list_sim = rc.zrange('RD_88475', 0, -1)
# data_final.loc[data['Ma san pham'].isin(list_sim)]

In [7]:
# #delete redis sorted set
# for x in range(len(df2['Ma san pham'])):
#         rc.delete(str('RD_ml_'+df2['Ma san pham'][x]))

for x in dataframe.columns.values.tolist():
        rc.delete(str('RD_ml_'+x))

In [9]:
#save to hash
# similar_tmp = {str(df2['Ma san pham'][x]): list_similar(df2['Ma san pham'][x], len(df2['Ma san pham']), dataframe) for x in range(len(df2['Ma san pham']))}
# rc.hmset('RD_recommender_maylanh', similar_tmp)
# rc.hgetall('RD_recommender_maylanh')
# rc.hmget('RD_recommender_maylanh', "67769", "68784", '75457')

[None, None, None]

# best sale items

In [82]:
db = pd.read_excel('/home/tungrd/TA/recommenderr/db.xlsx', sheetname='db', header = 0, index_col=0, convert_float= True)

In [80]:
# test
str(np.int(db.iloc[1][5]))
db.isnull().values.any()
db.isnull().sum()

'73693'

In [284]:
db = db.fillna(value= 0)

In [247]:
db['cate']= db['cate'].apply(lambda x: str(np.int(x)))
db['Web id']= db['Web id'].apply(lambda x: str(np.int(x)))

In [250]:
#table = db.pivot_table(df, index = ['cate', 'Web id'], aggfunc= {'Web id':'count'})
#table.query('cate == [42]')

In [286]:
best_sale_frame = db['Web id'].groupby([db['cate'], db['Web id']]).count().unstack()
best_sale_frame = best_sale_frame.fillna(value=0)
best_sale_frame

In [290]:
# get list best sale items
best_tmp = best_sale_frame.loc['42'].sort_values(ascending = False, inplace = False).index.tolist()[0: 10]
# 
best_tmp
best_sale_frame.head(10)

In [306]:
best_sale = list_best_sale('1992',20, best_sale_frame)

In [315]:
best_sale1 = list_best_sale('42',20, best_sale_frame)

In [318]:
list_index = best_sale.index.tolist()
list_index1 = best_sale1.index.tolist()

In [319]:
a = list(set(list_index) & set(list_index1))

# Connect to Redis

In [14]:
import redis
from rediscluster import StrictRedisCluster
# Requires at least one node for cluster discovery. Multiple nodes is recommended.
startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)
#!redis-cli -h 10.1.12.30 -p 16379 -c
#rc.dbsize()

In [1]:
#Add to redis
for x in range(len(best_sale_frame.index)):
    a = list_best_sale(best_sale_frame.index[x], 20, best_sale_frame)
    for y in range(20):
        rc.zadd(str('RD_ml_sale_'+best_sale_frame.index[x]), np.int(a.values[y]), str(a.index[y]))

In [313]:
rc.zrangebyscore('RD_ml_sale_1992', 0, 1)

[]

In [302]:
#delete redis sorted set
for x in range(len(best_sale_frame.index)):
        rc.delete(str('RD_ml_sale'+best_sale_frame.index[x]))

# recommended by product feature

In [11]:
# data_final[data_final.Category == '42'].Ma_san_pham.tolist()

In [46]:
def RDprofeature(dataframe, product_id = '74110', numofpro = 10, show = 0): 
    product = list(data['Ten san pham'][data['Ma san pham'] == product_id])
    a = list_similar(product_id, numofpro, dataframe)
    a_value = map(lambda x: np.str(x), list(a.index))#product_id
    a_score = map(lambda x: float(x), list(a.values))
    df_test = df2.loc[data['Ma san pham'].isin(a_value)]
    df_test = df_test.sort_values('Ma san pham')
    a = a.to_frame()
    # a.sort_index()
    a['Ma san pham']= a.index
    a = a.sort('Ma san pham')

    df_name = data[['Ma san pham', 'Ten san pham', 'Hang san xuat', 'Gia']][data['Ma san pham'].isin(a_value)]
    df_name = df_name.sort('Ma san pham')
    df_test = df_test.sort('Ma san pham')
    df_test['Ten san pham'] = df_name['Ten san pham']
    df_test['Ma san pham'] = df_name['Ma san pham']
    df_test['Gia'] = df_name['Gia']
    df_test['Score'] = list(a[product_id])
    df_test = df_test.sort('Score')
    if show == 0:
        return df_test
    elif show == 1:
        print('san pham: {} - {}\n'.format(product_id, product))
        return df_test[['Ten san pham', 'Ma san pham', 'Gia', 'Score']]
    else:
        return df_test

In [10]:
# RDprofeature(dataframe_42, '87839',4, 1)

In [11]:
# data[['Ten san pham', 'Gia']][data['Ma san pham'] == '87839']

In [12]:
# data

#     Testing Distance

In [265]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

In [266]:
# Testing Distance
# dt = data_final[data_final.Category == '42']
# ol = data_final[data_final.Category=='60']
# fuzz_data = pd.DataFrame(index = dt['Ma_san_pham'].unique(), columns= ol['Ma_san_pham'].unique())
# #asign values to dataframe
# for dth in range(fuzz_data.shape[0]):
#     for olu in range(fuzz_data.shape[1]):
#         fuzz_data.iloc[dth][olu] = fuzz.partial_ratio(dt.Ten_san_pham.iloc[dth], ol.Ten_san_pham.iloc[olu])

# #for x in fuzz_data.columns.values.tolist():86689
# fuzz_data.loc["70258"].sort(inplace = False)[-1:].values[0] > 68        

# fuzz_data.loc['70258'].sort(inplace = False)[-1:].index.tolist()
# ol.Ten_san_pham[ol.Ma_san_pham.isin(fuzz_data.loc['70258'].sort(inplace = False)[-1:].index.tolist())].values.tolist()[0]

# dt['Op_lung'] = ''
# for item in dt.Ma_san_pham.tolist():
#     if fuzz_data.loc[item].sort(inplace = False)[-1:].values[0]> 60:
#         dt.Op_lung[dt['Ma_san_pham'] == item] = ol.Ten_san_pham[ol.Ma_san_pham.isin(fuzz_data.loc[item].sort(inplace = False)[-1:].index.tolist())].values[0]
#     else:
#         dt.Op_lung[dt['Ma_san_pham'] == item] = ""
        

# #ol[ol.Ma_san_pham.isin(['91633', '90287', '91713','91635'])]
# ol.Ten_san_pham[ol.Ma_san_pham.isin(['91585',
# '91535',
# '90285',
# '90939',
# '89671',
# '89669',
# '90287',
# '90368',
# '89482',
# '90592',
# '91638',
# '90369',
# '90938',
# '91713',
# '91388',
# '91568',
# '89673',
# '89712',
# '89597'])]        

# Create table to save into DB

In [749]:
# Just test, I think it not work...
df = data[data['Category'].isin(['42', '44', '54','55','57','58'])]
#drop all those columns like (value, value2, value3) where all rows have same values
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

In [750]:
df.shape

(596, 180)

In [802]:
bf = db_test1

In [804]:
bf['Ma san pham']=bf['Ma san pham'].apply(lambda x: int(x))

In [7]:
# set index for dataframe
bf = bf.set_index(bf['Ma san pham'])
del bf['Ma san pham']
# bf.head()

In [806]:
# normalize value to [0;1]
bf -= bf.min()
bf /= bf.max()

In [807]:
b =pd.DataFrame(bf.unstack(level= -3))

In [808]:
type(b)

pandas.core.frame.DataFrame

In [809]:
b.shape

(26360, 1)

In [810]:
b = b.reset_index()

In [811]:
# convert string to np.nan
b = b.applymap(lambda x: pd.to_numeric(x, errors='coerce') if(x =='nan') else x)
# drop value is np.nan
b = b.dropna(axis = 0, how= 'any')

In [812]:
b = b.icol([1, 0 , 2])

In [813]:
# set name for columns
b.columns = ['ProductID', 'Property', 'Value']

In [8]:
b[b['ProductID'] == 77866]

In [819]:
#write to excel
ew = pd.ExcelWriter('/home/tungrd/TA/recommenderr/save_db_test.xlsx', options={'encoding':'utf-8'})
b.to_excel(ew, 'save_db_test')
ew.save()

# #read file excel
# dataframe = pd.read_excel('/home/tungrd/TA/recommenderr/thesameall.xlsx', sheetname='thesameall', header = 0, index_col=0 )

In [867]:
bf.shape

(659, 40)

In [9]:
#calculate the similarity for special CATE

dataframe = pd.DataFrame(index = bf.index, columns= [np.arange(bf.shape[0])])
for a in range(bf.shape[0]):
    for b in range(df2.shape[0]):
        dataframe.iloc[a][b] = similar(bf.iloc[a][:], bf.iloc[b][:])#length3
        
dataframe.columns = bf.index

In [936]:
# length3(bf.loc[87649], bf.loc[60677])
similar(bf.loc[87649], bf.loc[60677])

0.27623934040049253

In [10]:
#list_similar(id_produc, number_of_recommended_product)
a = dataframe[75180].sort(inplace = False)[: 10]
#a= dataframe_['76636'].sort(inplace = False)[:10]
#review (change df2 or data is name of dataframe)
a_value = map(lambda x: np.str(x), list(a.index))
a_value = [int(x) for x in a_value]
db_test1.loc[db_test1['Ma san pham'].isin(a_value)]
# data_final.loc[data['Ma san pham'].isin(a_value)]

In [939]:
a_value = [int(x) for x in a_value]
a_value